<a href="https://colab.research.google.com/github/nichmars/Test/blob/main/AttemptAtPercentChange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import io
import pandas as pd

# This line will prompt you to select and upload a file from your local machine
uploaded = files.upload()

# Get the name of the uploaded file (assuming only one file is uploaded)
file_name = next(iter(uploaded))

df = pd.read_excel(file_name)



Saving Filtered Data with Subtypes Again (1).xlsx to Filtered Data with Subtypes Again (1) (1).xlsx


In [30]:
df = pd.read_excel(file_name)

# List of metrics we want to analyze
metrics = ["Accounts", "Contributions", "Grants", "Assets"]

# Calculate the difference in years for each 'ein' to check for consecutive years
df = df.sort_values(["ein", "Year"])
df["year_diff"] = df.groupby("ein")["Year"].diff()


# Loop through each metric to calculate percent change and predicted values (ignoring non-consecutive years)
for m in metrics:
    pct_col = f"{m.lower()}_pct_change"

    df[pct_col] = df.groupby("ein")[m].pct_change()
    df.loc[df["year_diff"] != 1, pct_col] = None


#df.head(30)

In [31]:
# Compute median growth for each metric by Year, Type, and subtype
median_growth = (
    df
    .groupby(["Year", "Type", "subtype"])
    [[f"{m.lower()}_pct_change" for m in metrics]]
    .median()
    .reset_index()
)

# Rename columns to indicate these are median growth values
median_growth = median_growth.rename(
    columns={
        f"{m.lower()}_pct_change": f"median_{m.lower()}_growth"
        for m in metrics
    }
)

# Merge the median growth values back into the original DataFrame
df = df.merge(
    median_growth,
    on=["Year", "Type", "subtype"],
    how="left"
)

# Use median growth to predict the next year's value for each metric
for m in metrics:
    df[f"predicted_{m.lower()}"] = (
        df[m] * (1 + df[f"median_{m.lower()}_growth"])
    )

predictions = df.copy()
predictions["Year"] = predictions["Year"] + 1

In [32]:
df.head()

,ein,Year,Accounts,Contributions,Grants,Assets,Type,subtype,year_diff,accounts_pct_change,...,grants_pct_change,assets_pct_change,median_accounts_growth,median_contributions_growth,median_grants_growth,median_assets_growth,predicted_accounts,predicted_contributions,predicted_grants,predicted_assets
0,10391479,2007,270,13347384.0,10628512,95282896.0,Community Foundation,Community,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10391479,2008,276,8389212.0,7419626,68897739.0,Community Foundation,Community,1.0,0.022222,...,-0.301913,-0.276914,0.048810,-0.205356,0.181462,-0.177510,289.471429,6.666438e+06,8.766008e+06,5.666769e+07
2,10391479,2009,278,10016375.0,8857052,83057576.0,Community Foundation,Community,1.0,0.007246,...,0.193733,0.205520,0.026549,-0.210061,-0.142366,0.133582,285.380531,7.912323e+06,7.596112e+06,9.415256e+07
3,10391479,2010,284,14088617.0,7569699,97807493.0,Community Foundation,Community,1.0,0.021583,...,-0.145348,0.177587,0.029986,0.252868,0.099772,0.112722,292.515914,1.765117e+07,8.324945e+06,1.088325e+08
4,10391479,2011,296,19783555.0,9802671,106056922.0,Community Foundation,Community,1.0,0.042254,...,0.294988,0.084344,0.042866,0.112210,0.153378,0.043404,308.688304,2.200347e+07,1.130619e+07,1.106602e+08


In [33]:
df.to_excel(
    "Attempt_at_Imputing.xlsx",
    index=False
)



In [58]:
import numpy as np

# Convert predicted columns to numeric and replace invalid values
pred_cols = ["predicted_accounts", "predicted_contributions",  "predicted_grants", "predicted_assets"]

for col in pred_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.replace([np.inf, -np.inf], np.nan)

# Function to calculate relative error for a column
def relative_error(actual_col, predicted_col):
    valid_rows = df[df[predicted_col].notna()]
    total_actual = valid_rows[actual_col].sum()
    total_predicted = valid_rows[predicted_col].sum()
    RE = (total_actual - total_predicted) / total_actual
    return RE

# Compute relative errors
RE_account = relative_error("Accounts", "predicted_accounts")
RE_contributions = relative_error("Contributions", "predicted_contributions")
RE_grants = relative_error("Grants", "predicted_grants")
RE_assets = relative_error("Assets", "predicted_assets")

print("Relative Error - Account:", abs(RE_account))
print("Relative Error - Contributions:", abs(RE_contributions))
print("Relative Error - Grants:", abs(RE_grants))
print("Relative Error - Assets:", abs(RE_assets))


Relative Error - Account: 0.08151860031249465
Relative Error - Contributions: 0.15853841697327792
Relative Error - Grants: 0.14422491684602598
Relative Error - Assets: 0.14010277548647931
